# Analysis of synchronized freezing
# How to analye FreezeFrame video (Win10)
    2019/12/26 Bug fix: we need to install win64 ffmpeg
    
    1) Video analysis manually. Write start and stop time as cvs file.
    2) Read entire cvs file which is not listed master table.
    3) Analysis.
    
---
## Step 1. Export AVI file using FreezeFrame Viewer
    1) install FreezeFrameInstaller4104.zip
    2) in FreezeFrame Viewer
        Video/Compress Image Files, as AVI file using MJPEG Compressor
---
## Step 2. Conversion avi to mp4
### Rotate and compress all avi videos in the current directory
    Provide current directory

In [1]:
# Ubuntu path
#path = '/home/alexei/ubg-tera05/WD_Passport/Alexei/synchrony/110919'
# Windows path
path = r'Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419'
path = r'Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi'
print(path)

Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi


In [2]:
import os
import ffmpeg

#path = os.getcwd()
#print(path)

os.chdir(path)
print(os.listdir(path))

for file in os.listdir(path):
    base = os.path.splitext(file)[0]
    extn = os.path.splitext(file)[1]
    if extn == '.avi':
        # !ffmpeg -i {file} -vf "transpose=2,transpose=2" -vcodec libx264 {base}.mp4
        fileIN = os.path.join(path,file)
        print('file = ', fileIN, end = ' ')
        stream = ffmpeg.input(fileIN)
        stream = ffmpeg.hflip(stream)
        stream = ffmpeg.vflip(stream)
        fileOUT = os.path.join(path,base) + '.mp4'
        print('---> output = ', fileOUT)
        stream = ffmpeg.output(stream, fileOUT, vcodec='libx264')
        ffmpeg.run(stream)

print('end')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Z:\\wataru\\WD_Passport\\Alexei\\synchrony\\2019_9_12\\122419_wi'

import ffmpeg
stream = ffmpeg.input('input.mp4')
stream = ffmpeg.hflip(stream)
stream = ffmpeg.output(stream, 'output.mp4')
ffmpeg.run(stream)

## Step 3. Video Analysis using VideoPlayer

In [1]:
# Orignated from maximus009/VideoPlayer
# https://github.com/maximus009/VideoPlayer

import os, sys, time
import cv2, numpy as np
import videoplay as vp

###################################
# Full path of the video file
#video = r'Z:\wataru\WD_Passport\Wataru\082216\m24a.mp4'
path = r'Z:\wataru\WD_Passport\Wataru\082216'
video = 'm24a.mp4'
video = os.path.join(path,video)

###################################
# Initialize video windows
cv2.namedWindow('image')
cv2.moveWindow('image',250,150)
# Open video file
cap = cv2.VideoCapture(video)
# Get the total number of frame
tots = cap.get(cv2.CAP_PROP_FRAME_COUNT)
# Add two trackbars
cv2.createTrackbar('S','image', 0, int(tots)-1, vp.flick)
cv2.setTrackbarPos('S','image',0)
cv2.createTrackbar('F','image', 1, 100, vp.flick)
frame_rate = 30
cv2.setTrackbarPos('F','image',frame_rate)

###################################
# Initialize contol window
cv2.namedWindow('controls')
cv2.moveWindow('controls',250,50)

# Generate image (controls, 50 x 750) from text and display in the control window
controls = np.zeros((50,750),np.uint8)
cv2.putText(controls, "W/w: Play, S/s: Stop, A/a: Prev_Frame, D/d: Next_Frame, E/e: Faster, Q/q: Slower, Esc: Exit",
            (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.45, 255)
cv2.imshow("controls",controls)

###################################
status_list = { ord('s'):'stop', ord('S'):'stop',
                ord('w'):'play', ord('W'):'play',
                ord('a'):'prev_frame', ord('A'):'prev_frame',
                ord('d'):'next_frame', ord('D'):'next_frame',
                ord('q'):'slow', ord('Q'):'slow',
                ord('e'):'fast', ord('E'):'fast',
                ord('c'):'snap', ord('C'):'snap',
                -1:'no_key_press', 
                27:'exit'}
current_frame = 0
status = 'stop'
start_time = time.time()
realFrameRate = frame_rate

# Adjust video width 750 pixel
ret, im = cap.read()
video_format = im.shape
print("Video resolution: {}".format(video_format))
r = 750 / im.shape[1]
dim = (750, int(im.shape[0] * r))

###################################
# Main loop
while True:
    try:
        # If reach to the end, play from the begining
        if current_frame==tots-1:
            current_frame=0
        
        # read a frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        ret, im = cap.read()

        # put current state and real frame rate in the image
        im_text = status + " frame_rate: " + str(realFrameRate) + " fps"
        vp.add_text(im, im_text, 20, 0.5)        
        
        # resize video
        im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
        if im.shape[0]>600:
            im = cv2.resize(im, (500,500))
            controls = cv2.resize(controls, (im.shape[1],25))
            
        # show video frame
        cv2.imshow('image', im)
        
        # Read key input
        status_new = status_list[cv2.waitKey(1)]
        if status_new != 'no_key_press':
            status = status_new

        if status == 'play':
            frame_rate = cv2.getTrackbarPos('F','image')
            if frame_rate == 0.0:
                continue        
            if (time.time() - start_time) > 1.0/frame_rate:
                realFrameRate = round(1.0/(time.time() - start_time),2)
                current_frame+=1
                cv2.setTrackbarPos('S','image',current_frame)
                start_time = time.time()
                continue
        elif status == 'stop':
            current_frame = cv2.getTrackbarPos('S','image')
        elif status == 'exit':
            break
        elif status=='prev_frame':
            current_frame-=1
            cv2.setTrackbarPos('S','image',current_frame)
            status='stop'
        elif status=='next_frame':
            current_frame+=1
            cv2.setTrackbarPos('S','image',current_frame)
            status='stop'
        elif status=='slow':
            frame_rate = max(frame_rate - 1, 0)
            cv2.setTrackbarPos('F', 'image', frame_rate)
            status='play'
        elif status=='fast':
            frame_rate = min(100,frame_rate+1)
            cv2.setTrackbarPos('F', 'image', frame_rate)
            status='play'
        elif status=='snap':
            cv2.imwrite("./"+"Snap_"+str(i)+".jpg",im)
            print("Snap of Frame",current_frame,"Taken!")
            status='stop'

    except KeyError:
        print("Invalid Key was pressed")

# Clean up windows
cap.release()
cv2.destroyAllWindows()

Video resolution: (236, 304, 3)
Invalid Key was pressed


In [7]:
import cv2, numpy as np

length = 5

def draw_circle(event,x,y,flags,param):
    global mouseX,mouseY
    if event == cv2.EVENT_LBUTTONDBLCLK:
        # cv2.circle(img,(x,y),10,(255,0,0),-1)
        cv2.line(img,(x+length,y+length),(x-length,y-length),(255,0,0),2)
        cv2.line(img,(x+length,y-length),(x-length,y+length),(255,0,0),2)
        mouseX,mouseY = x,y
        
img = np.zeros((512,512,3), np.uint8)
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)

while(1):
    cv2.imshow('image',img)
    
    
    k = cv2.waitKey(20) & 0xFF
    if k == 27:
        break
    elif k == ord('a'):
        print(mouseX,mouseY)
        
cv2.destroyAllWindows()

In [15]:
# Orignated from maximus009/VideoPlayer
# https://github.com/maximus009/VideoPlayer

import os, sys, time
import cv2, numpy as np
import videoplay as vp

length = 5
x1,y1,x2,y2 = -1,-1,-1,-1
drag = False
sub = 'sub1'

def draw_circle(event,x,y,flags,param):
    global x1,y1,x2,y2,drag,sub
    if event == cv2.EVENT_LBUTTONDOWN:
        drag = True
        sub = 'sub1'
        x1,y1 = x,y  
    elif event == cv2.EVENT_LBUTTONUP:
        drag = False
        x1,y1 = -1,-1
    elif event == cv2.EVENT_RBUTTONDOWN:
        drag = True
        sub = 'sub2'
        x2,y2 = x,y
    elif event == cv2.EVENT_RBUTTONUP:
        drag = False
        x2,y2 = -1,-1
    elif event == cv2.EVENT_MOUSEMOVE:
        if drag == True:
            if sub == 'sub1':
                x1,y1 = x,y
            else:
                x2,y2 = x,y
        
###################################
# Full path of the video file
#video = r'Z:\wataru\WD_Passport\Wataru\082216\m24a.mp4'
path = r'Z:\wataru\WD_Passport\Wataru\082216'
video = 'm24a.mp4'
video = os.path.join(path,video)

###################################
# Initialize video windows
cv2.namedWindow('image')
cv2.moveWindow('image',250,150)
# Set mouse callback
cv2.setMouseCallback('image',draw_circle)
# Open video file
cap = cv2.VideoCapture(video)
# Get the total number of frame
tots = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Add two trackbars
cv2.createTrackbar('S','image', 0, int(tots)-1, vp.flick)
cv2.setTrackbarPos('S','image',0)
cv2.createTrackbar('F','image', 1, 100, vp.flick)
frame_rate = 30
cv2.setTrackbarPos('F','image',frame_rate)

###################################
# Initialize contol window
#cv2.namedWindow('controls')
#cv2.moveWindow('controls',250,50)

# Generate image (controls, 50 x 750) from text and display in the control window
#controls = np.zeros((50,750),np.uint8)
#cv2.putText(controls, "W/w: Play, S/s: Stop, A/a: Prev_Frame, D/d: Next_Frame, E/e: Faster, Q/q: Slower, Esc: Exit",
#            (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.45, 255)
#cv2.imshow("controls",controls)

###################################
status_list = { ord('s'):'stop',
                ord('w'):'play',
                ord('a'):'prev_frame', ord('d'):'next_frame',
                ord('q'):'slow', ord('e'):'fast',
                ord(' '):'snap',
                -1:'no_key_press', 
                27:'exit'}
current_frame = 0
status = 'stop'
start_time = time.time()
realFrameRate = frame_rate

# Adjust video width 750 pixel
ret, im = cap.read()
video_format = im.shape
print("Video resolution: {}".format(video_format))
x_pixcels = im.shape[1]
y_pixcels = im.shape[0]
r = 3
dim = (x_pixcels*r, y_pixcels*r)

# prepare to store trajectory 
xy1 = np.array([[-1 for x in range(tots)] for y in range(2)])
xy2 = np.array([[-1 for x in range(tots)] for y in range(2)])
###################################
# Main loop
while True:
    try:
        # If reach to the end, play from the begining
        if current_frame==tots-1:
            current_frame=0
        
        # read a frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        ret, im = cap.read()

     
        
        # resize video
        im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
#         if im.shape[0]>600:
#             im = cv2.resize(im, (500,500))
#             controls = cv2.resize(controls, (im.shape[1],25))

        # put current state and real frame rate in the image
        im_text = "video_status: " + status + ", frame_rate: " + str(realFrameRate) + " fps"
        vp.add_text(im, im_text, 20, 0.5)   

        if x1 !=-1:
            xy1[:,current_frame] = [x1,y1]
        if x2 !=-1:
            xy2[:,current_frame] = [x2,y2]            

        if xy1[0,current_frame] != -1:
            [_x1,_y1] = xy1[:,current_frame]
            cv2.line(im,(_x1+length,_y1+length),(_x1-length,_y1-length),(0,0,255),2)
            cv2.line(im,(_x1+length,_y1-length),(_x1-length,_y1+length),(0,0,255),2)   
        if xy2[0,current_frame] != -1:
            [_x2,_y2] = xy2[:,current_frame]            
            cv2.line(im,(_x2+length,_y2+length),(_x2-length,_y2-length),(0,255,0),2)
            cv2.line(im,(_x2+length,_y2-length),(_x2-length,_y2+length),(0,255,0),2)              
        
        # show video frame
        cv2.imshow('image', im)
        
        # Read key input
        status_new = status_list[cv2.waitKey(1)]

        if status_new != 'no_key_press':
            status_pre = status
            status = status_new

        if status == 'play':
            frame_rate = cv2.getTrackbarPos('F','image')
            if frame_rate == 0.0:
                continue        
            if (time.time() - start_time) > 1.0/frame_rate:
                realFrameRate = round(1.0/(time.time() - start_time),2)
                current_frame+=1
                cv2.setTrackbarPos('S','image',current_frame)
                start_time = time.time()
                continue
        elif status == 'stop':
            current_frame = cv2.getTrackbarPos('S','image')
        elif status == 'exit':
            break
        elif status=='prev_frame':
            current_frame-=1
            cv2.setTrackbarPos('S','image',current_frame)
            status='stop'
        elif status=='next_frame':
            current_frame+=1
            cv2.setTrackbarPos('S','image',current_frame)
            status='stop'
        elif status=='slow':
            frame_rate = max(frame_rate - 1, 0)
            cv2.setTrackbarPos('F', 'image', frame_rate)
            status=status_pre
        elif status=='fast':
            frame_rate = min(100,frame_rate+1)
            cv2.setTrackbarPos('F', 'image', frame_rate)
            status=status_pre
        elif status=='snap':
            cv2.imwrite("./"+"Snap_"+str(i)+".jpg",im)
            print("Snap of Frame",current_frame,"Taken!")
            status='stop'

    except KeyError:
        print("Invalid Key was pressed")
# Clean up windows
cap.release()
cv2.destroyAllWindows()

Video resolution: (236, 304, 3)


In [9]:
# Orignated from maximus009/VideoPlayer
# https://github.com/maximus009/VideoPlayer

import os, sys, time
import cv2, numpy as np
import videoplay as vp
import math
        
###################################
# Full path of the video file
#video = r'Z:\wataru\WD_Passport\Wataru\082216\m24a.mp4'
path = r'Z:\wataru\WD_Passport\Wataru\082216'
video = 'm24a.mp4'
video = os.path.join(path,video)

###################################
# Initialize video windows
cv2.namedWindow('image')
cv2.moveWindow('image',250,150)
# Set mouse callback
cv2.setMouseCallback('image',dragging)
# Open video file
cap = cv2.VideoCapture(video)
# Get the total number of frame
tots = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Add two trackbars
cv2.createTrackbar('S','image', 0, int(tots)-1, vp.flick)
cv2.setTrackbarPos('S','image',0)
cv2.createTrackbar('F','image', 1, 100, vp.flick)
frame_rate = 30
cv2.setTrackbarPos('F','image',frame_rate)

###################################
# Initialize contol window
#cv2.namedWindow('controls')
#cv2.moveWindow('controls',250,50)

# Generate image (controls, 50 x 750) from text and display in the control window
#controls = np.zeros((50,750),np.uint8)
#cv2.putText(controls, "W/w: Play, S/s: Stop, A/a: Prev_Frame, D/d: Next_Frame, E/e: Faster, Q/q: Slower, Esc: Exit",
#            (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.45, 255)
#cv2.imshow("controls",controls)

###################################
status_list = { ord('s'):'stop',
                ord('w'):'play',
                ord('a'):'prev_frame', ord('d'):'next_frame',
                ord('q'):'slow', ord('e'):'fast',
                ord(' '):'snap',
                ord('0'):'drag_mode',ord('1'):'click_mode_sub1',ord('2'):'click_mode_sub2',
                -1:'no_key_press', 
                27:'exit'}
current_frame = 0
status = 'stop'
start_time = time.time()
realFrameRate = frame_rate

# Adjust video width 750 pixel
ret, im = cap.read()
video_format = im.shape
print("Video resolution: {}".format(video_format))
x_pixcels = im.shape[1]
y_pixcels = im.shape[0]
r = 3
dim = (x_pixcels*r, y_pixcels*r)

length = 5 # cross cursor length

# Mouse events handler (global)
x1,y1,x2,y2 = 100,100,120,100
drag = False
click = False
sub = 'sub1'
pixel_limit = 10.0
mode = 'click_mode_sub1'

# prepare to store trajectory 
xy1 = np.array([[-1 for x in range(tots)] for y in range(2)])
xy2 = np.array([[-1 for x in range(tots)] for y in range(2)])
###################################
# Main loop
while True:
    try:
        # If reach to the end, play from the begining
        if current_frame==tots-1:
            current_frame=0
        
        # read a frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        ret, im = cap.read()
        
        # resize video
        im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)

        # put current state and real frame rate in the image
        im_text = "video_status: " + status + ", frame_rate: " + \
                str(realFrameRate) + " fps, mode: " + mode
                
        vp.add_text(im, im_text, 20, 0.5)   

        # display cursors
        if xy1[0,current_frame] == -1 or drag or click:
            xy1[:,current_frame] = [x1,y1]        
        [_x1,_y1] = xy1[:,current_frame]
        cv2.line(im,(_x1+length,_y1+length),(_x1-length,_y1-length),(0,255,0),2)
        cv2.line(im,(_x1+length,_y1-length),(_x1-length,_y1+length),(0,255,0),2)
        
        if xy2[0,current_frame] == -1 or drag or click:
            xy2[:,current_frame] = [x2,y2]      
        [_x2,_y2] = xy2[:,current_frame]            
        cv2.line(im,(_x2+length,_y2+length),(_x2-length,_y2-length),(0,0,255),2)
        cv2.line(im,(_x2+length,_y2-length),(_x2-length,_y2+length),(0,0,255),2)              
            
        # show video frame
        cv2.imshow('image', im)
        
        # Read key input
        status_new = status_list[cv2.waitKey(1)]

        if status_new != 'no_key_press':
            status_pre = status
            status = status_new

        if status == 'play':
            frame_rate = cv2.getTrackbarPos('F','image')
            if frame_rate == 0.0:
                continue        
            if (time.time() - start_time) > 1.0/frame_rate:
                realFrameRate = round(1.0/(time.time() - start_time),2)
                current_frame+=1
                cv2.setTrackbarPos('S','image',current_frame)
                start_time = time.time()
                continue
        elif status == 'stop':
            current_frame = cv2.getTrackbarPos('S','image')
        elif status == 'exit':
            break
        elif status=='prev_frame':
            current_frame-=1
            cv2.setTrackbarPos('S','image',current_frame)
            status='stop'
        elif status=='next_frame':
            current_frame+=1
            cv2.setTrackbarPos('S','image',current_frame)
            status='stop'
        elif status=='slow':
            frame_rate = max(frame_rate - 1, 0)
            cv2.setTrackbarPos('F', 'image', frame_rate)
            status=status_pre
        elif status=='fast':
            frame_rate = min(100,frame_rate+1)
            cv2.setTrackbarPos('F', 'image', frame_rate)
            status=status_pre
        elif status=='drag_mode':
            mode = 'drag_mode'
            status=status_pre
        elif status=='click_mode_sub1':
            mode = 'click_mode_sub1'
            status=status_pre
        elif status=='click_mode_sub2':
            mode = 'click_mode_sub2'
            status=status_pre
        elif status=='snap':
            cv2.imwrite("./"+"Snap_"+str(i)+".jpg",im)
            print("Snap of Frame",current_frame,"Taken!")
            status='stop'

    except KeyError:
        print("Invalid Key was pressed")
# Clean up windows
cap.release()
cv2.destroyAllWindows()

###################################
# Mouse events handler
def dragging(event,x,y,flags,param):
    global x1,y1,x2,y2,drag,sub,click
    if event == cv2.EVENT_LBUTTONDOWN:
        if mode == 'drag_mode':
            if abs(math.sqrt(x1**2+y1**2) - math.sqrt(x**2+y**2)) < pixel_limit:
                drag = True
                sub = 'sub1'            
                x1,y1 = x,y          
            if abs(math.sqrt(x2**2+y2**2) - math.sqrt(x**2+y**2)) < pixel_limit:
                drag = True
                sub = 'sub2'            
                x2,y2 = x,y
        elif mode[0:-1] =='click_mode_sub':
            click = True
            if mode[-1] == '1':
                x1,y1 = x,y                
            if mode[-1] == '2':
                x2,y2 = x,y               
    elif event == cv2.EVENT_LBUTTONUP:
        drag = False
        sub = ''
        click = False
    elif event == cv2.EVENT_MOUSEMOVE:
        if drag == True:
            if sub == 'sub1':
                x1,y1 = x,y
            elif sub == 'sub2':
                x2,y2 = x,y


Video resolution: (236, 304, 3)


In [9]:
# Orignated from maximus009/VideoPlayer
# https://github.com/maximus009/VideoPlayer
import os, sys, time
import cv2, numpy as np
import videoplay as vp
import math

def video_cursor(video):
    ###################################
    # Full path of the video file
    #video = r'Z:\wataru\WD_Passport\Wataru\082216\m24a.mp4'
    path = r'Z:\wataru\WD_Passport\Wataru\082216'
    video = 'm24a.mp4'
    video = os.path.join(path,video)

    ###################################
    # Initialize video windows
    cv2.namedWindow('image')
    cv2.moveWindow('image',250,150)
    # Set mouse callback
    cv2.setMouseCallback('image',dragging)
    # Open video file
    cap = cv2.VideoCapture(video)
    # Get the total number of frame
    tots = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Add two trackbars
    cv2.createTrackbar('S','image', 0, int(tots)-1, vp.flick)
    cv2.setTrackbarPos('S','image',0)
    cv2.createTrackbar('F','image', 1, 100, vp.flick)
    frame_rate = 30
    cv2.setTrackbarPos('F','image',frame_rate)

    ###################################
    # Initialize contol window
    #cv2.namedWindow('controls')
    #cv2.moveWindow('controls',250,50)

    # Generate image (controls, 50 x 750) from text and display in the control window
    #controls = np.zeros((50,750),np.uint8)
    #cv2.putText(controls, "W/w: Play, S/s: Stop, A/a: Prev_Frame, D/d: Next_Frame, E/e: Faster, Q/q: Slower, Esc: Exit",
    #            (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.45, 255)
    #cv2.imshow("controls",controls)

    ###################################
    status_list = { ord('s'):'stop',
                    ord('w'):'play',
                    ord('a'):'prev_frame', ord('d'):'next_frame',
                    ord('q'):'slow', ord('e'):'fast',
                    ord(' '):'snap',
                    ord('0'):'drag_mode',ord('1'):'click_mode_sub1',ord('2'):'click_mode_sub2',
                    -1:'no_key_press', 
                    27:'exit'}
    current_frame = 0
    status = 'stop'
    start_time = time.time()
    realFrameRate = frame_rate

    # Adjust video width 750 pixel
    ret, im = cap.read()
    video_format = im.shape
    print("Video resolution: {}".format(video_format))
    x_pixcels = im.shape[1]
    y_pixcels = im.shape[0]
    r = 3
    dim = (x_pixcels*r, y_pixcels*r)

    length = 5 # cross cursor length

    # Mouse events handler (global)
    x1,y1,x2,y2 = 100,100,120,100
    drag = False
    click = False
    sub = 'sub1'
    pixel_limit = 10.0
    mode = 'click_mode_sub1'

    # prepare to store trajectory 
    xy1 = np.array([[-1 for x in range(tots)] for y in range(2)])
    xy2 = np.array([[-1 for x in range(tots)] for y in range(2)])
    ###################################
    # Main loop
    while True:
        try:
            # If reach to the end, play from the begining
            if current_frame==tots-1:
                current_frame=0

            # read a frame
            cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
            ret, im = cap.read()

            # resize video
            im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)

            # put current state and real frame rate in the image
            im_text = "video_status: " + status + ", frame_rate: " + \
                    str(realFrameRate) + " fps, mode: " + mode

            vp.add_text(im, im_text, 20, 0.5)   

            # display cursors
            if xy1[0,current_frame] == -1 or drag or click:
                xy1[:,current_frame] = [x1,y1]        
            [_x1,_y1] = xy1[:,current_frame]
            cv2.line(im,(_x1+length,_y1+length),(_x1-length,_y1-length),(0,255,0),2)
            cv2.line(im,(_x1+length,_y1-length),(_x1-length,_y1+length),(0,255,0),2)

            if xy2[0,current_frame] == -1 or drag or click:
                xy2[:,current_frame] = [x2,y2]      
            [_x2,_y2] = xy2[:,current_frame]            
            cv2.line(im,(_x2+length,_y2+length),(_x2-length,_y2-length),(0,0,255),2)
            cv2.line(im,(_x2+length,_y2-length),(_x2-length,_y2+length),(0,0,255),2)              

            # show video frame
            cv2.imshow('image', im)

            # Read key input
            status_new = status_list[cv2.waitKey(1)]

            if status_new != 'no_key_press':
                status_pre = status
                status = status_new

            if status == 'play':
                frame_rate = cv2.getTrackbarPos('F','image')
                if frame_rate == 0.0:
                    continue        
                if (time.time() - start_time) > 1.0/frame_rate:
                    realFrameRate = round(1.0/(time.time() - start_time),2)
                    current_frame+=1
                    cv2.setTrackbarPos('S','image',current_frame)
                    start_time = time.time()
                    continue
            elif status == 'stop':
                current_frame = cv2.getTrackbarPos('S','image')
            elif status == 'exit':
                break
            elif status=='prev_frame':
                current_frame-=1
                cv2.setTrackbarPos('S','image',current_frame)
                status='stop'
            elif status=='next_frame':
                current_frame+=1
                cv2.setTrackbarPos('S','image',current_frame)
                status='stop'
            elif status=='slow':
                frame_rate = max(frame_rate - 1, 0)
                cv2.setTrackbarPos('F', 'image', frame_rate)
                status=status_pre
            elif status=='fast':
                frame_rate = min(100,frame_rate+1)
                cv2.setTrackbarPos('F', 'image', frame_rate)
                status=status_pre
            elif status=='drag_mode':
                mode = 'drag_mode'
                status=status_pre
            elif status=='click_mode_sub1':
                mode = 'click_mode_sub1'
                status=status_pre
            elif status=='click_mode_sub2':
                mode = 'click_mode_sub2'
                status=status_pre
            elif status=='snap':
                cv2.imwrite("./"+"Snap_"+str(i)+".jpg",im)
                print("Snap of Frame",current_frame,"Taken!")
                status='stop'

        except KeyError:
            print("Invalid Key was pressed")
    # Clean up windows
    cap.release()
    cv2.destroyAllWindows()

    return

###################################
# Mouse events handler
def dragging(event,x,y,flags,param):
    global x1,y1,x2,y2,drag,sub,click
    if event == cv2.EVENT_LBUTTONDOWN:
        if mode == 'drag_mode':
            if abs(math.sqrt(x1**2+y1**2) - math.sqrt(x**2+y**2)) < pixel_limit:
                drag = True
                sub = 'sub1'            
                x1,y1 = x,y          
            if abs(math.sqrt(x2**2+y2**2) - math.sqrt(x**2+y**2)) < pixel_limit:
                drag = True
                sub = 'sub2'            
                x2,y2 = x,y
        elif mode[0:-1] =='click_mode_sub':
            click = True
            if mode[-1] == '1':
                x1,y1 = x,y                
            if mode[-1] == '2':
                x2,y2 = x,y               
    elif event == cv2.EVENT_LBUTTONUP:
        drag = False
        sub = ''
        click = False
    elif event == cv2.EVENT_MOUSEMOVE:
        if drag == True:
            if sub == 'sub1':
                x1,y1 = x,y
            elif sub == 'sub2':
                x2,y2 = x,y


Video resolution: (236, 304, 3)


In [1]:
import videoplay as vp

video = r'Z:\wataru\WD_Passport\Wataru\082216\m24a.mp4'
vp.video_cursor(video)

Video resolution: (236, 304, 3)


In [1]:
import cv2, numpy as np
tots = 10
xy1 = np.array([[-1 for x in range(tots)] for y in range(2)])

In [2]:
xy1

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [2]:
import numpy as np
import cv2

# Create a black image
img = np.zeros((512,512,3), np.uint8)

# Draw a diagonal blue line with thickness of 5 px
img = cv2.line(img,(0,0),(511,511),(255,0,0),5)

img = cv2.rectangle(img,(384,0),(510,128),(0,255,0),3)

img = cv2.circle(img,(447,63), 63, (0,0,255), -1)

img = cv2.ellipse(img,(256,256),(100,50),0,0,180,255,-1)

pts = np.array([[10,5],[20,30],[70,20],[50,10]], np.int32)
pts = pts.reshape((-1,1,2))
img = cv2.polylines(img,[pts],True,(0,255,255))

font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img,'OpenCV',(10,500), font, 4,(255,255,255),2,cv2.LINE_AA)


array([[[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]]

### Experimental GUI
    Not ready to use
   

In [8]:
import tkinter as tk

def cbc(id, tex):
    return lambda : callback(id, tex)

def callback(id, tex):
    s = 'At {} f is {}\n'.format(id, id**id/0.987)
    tex.insert(tk.END, s)
    tex.see(tk.END)             # Scroll if necessary

top = tk.Tk()
tex = tk.Text(master=top)
tex.pack(side=tk.RIGHT)
bop = tk.Frame()
bop.pack(side=tk.LEFT)
for k in range(1,10):
    tv = 'Say {}'.format(k)
    b = tk.Button(bop, text=tv, command=cbc(k, tex))
    b.pack()

tk.Button(bop, text='Exit', command=top.destroy).pack()
top.mainloop()